In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import psycopg2
import pandas as pd
import psycopg2.extras
import seaborn as sns
import numpy as np

In [6]:
import matplotlib.pyplot as plt

In [7]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="healthdb",
                                           user = "postgres",
                                           password = "ahanaf12",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [8]:
con = PostgresConnection().getConnection()

successfully connected to database


# Decision 1: Top 10 labs with the most assosciated tests

In [14]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT lab_name, test_count " \
              "FROM (" \
                "SELECT s.lab_key, s.lab_name, COUNT(DISTINCT p.test_name) AS test_count, " \
                "RANK() OVER (ORDER BY COUNT(DISTINCT p.test_name) DESC) AS rank " \
                "FROM healthdb_schema.\"fact_table\" t " \
                "JOIN healthdb_schema.\"lab_dim\" s on s.lab_key=t.lab_key " \
                "JOIN healthdb_schema.\"test_dim\" p on p.test_key=t.test_key " \
                "GROUP BY s.lab_key, s.lab_name " \
               ") subquery " \
               "WHERE rank <= 10 " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()
records

df = pd.DataFrame(records, columns=["Lab Name", "No. of Tests"]) 
df = df.set_index(['Lab Name'])

highest_tests_lab = df 
highest_tests_lab

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,No. of Tests
Lab Name,
Biochemistry,132
Immunology,112
Microbiology,54
Clinical Pathology,52
Serology,40
HISTO-PATHOLOGY,22
Haematology,21
BLOOD TRANSFUSION,15
CYTO-PATHOLOGY,14


# Decision 2: Find patient constitution (geography) during input year

In [36]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

year = 2020 # input("Year: ") 

select_stmt = "SELECT division, district, patient_count " \
              "FROM (" \
                "SELECT p.division, p.district, COUNT(t.pik) AS patient_count, " \
                "RANK() OVER (ORDER BY COUNT(t.pik) DESC) AS rank " \
                "FROM healthdb_schema.\"fact_table\" t " \
                "JOIN healthdb_schema.\"time_dim\" s on s.time_key=t.time_key " \
                "JOIN healthdb_schema.\"geography_dim\" p on p.geo_key=t.geo_key " \
                "WHERE s.year = {} " \
                "GROUP BY p.division, p.district " \
               ") subquery " \
               "ORDER BY division, rank".format(year, month, day_start, day_end) 
 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Division", "District", "Patient Count"]) 
df = df.set_index(['Division'])

patient_geography = df 
patient_geography

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,District,Patient Count
Division,,
BARISAL,PATUAKHALI,241
BARISAL,BHOLA,175
BARISAL,BARISAL,138
BARISAL,PIROJPUR,127
BARISAL,BARGUNA,107
...,...,...
RANGPUR,LALMONIRHAT,41
SYLHET,SUNAMGANJ,266
SYLHET,SYLHET,189


# Decision 3: Labwise Sales

In [29]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT lab_name, test_count, sales " \
              "FROM (" \
                "SELECT s.lab_name, COUNT(p.test_name) AS test_count, SUM(t.sales_price) AS sales, " \
                "RANK() OVER (ORDER BY SUM(t.sales_price) DESC) AS rank " \
                "FROM healthdb_schema.\"fact_table\" t " \
                "JOIN healthdb_schema.\"lab_dim\" s on s.lab_key=t.lab_key " \
                "JOIN healthdb_schema.\"test_dim\" p on p.test_key=t.test_key " \
                "GROUP BY s.lab_name " \
               ") subquery " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Lab Name", "Test Count", "Sales"]) 
df = df.set_index(['Lab Name'])

labwise_sales = df 
labwise_sales

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Test Count,Sales
Lab Name,,
Biochemistry,43206,35239170
Immunology,17664,28397040
Haematology,16078,6585010
Microbiology,3812,5554660
BLOOD TRANSFUSION,1067,4019880
Serology,3856,3581680
Clinical Pathology,5795,2185610
PCR LAB,130,724500
HISTO-PATHOLOGY,265,586320


# Decision 4: Types of distinct specimens a lab handles

In [35]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT lab_name, specimen, test_number " \
              "FROM (" \
                "SELECT s.lab_name, p.specimen, COUNT(DISTINCT p.test_key) AS test_number, " \
                "RANK() OVER (ORDER BY COUNT(DISTINCT p.test_key) DESC) AS rank " \
                "FROM healthdb_schema.\"fact_table\" t " \
                "JOIN healthdb_schema.\"lab_dim\" s on s.lab_key=t.lab_key " \
                "JOIN healthdb_schema.\"test_dim\" p on p.test_key=t.test_key " \
                "GROUP BY s.lab_name, p.specimen " \
               ") subquery " \
               "ORDER BY lab_name, rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Lab Name", "Specimen", "Tests Requiring Specimen"]) 
df = df.set_index(['Lab Name'])

distinct_specimens = df 
distinct_specimens

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Specimen,Tests Requiring Specimen
Lab Name,,
Biochemistry,BLOOD,70
Biochemistry,URINE,27
Biochemistry,FLUID,6
Biochemistry,ASCITIC FLUID,6
Biochemistry,BLOOD.,6
...,...,...
PHARMACOGENETICS LAB,WHOLE BLOOD,2
Serology,BLOOD,37
Serology,BLOOD.,1


# Decision 5: Year-wise busiest quarters

In [39]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT year, quarter, patient_count " \
              "FROM (" \
                "SELECT s.year, s.quarter, COUNT(t.pik) AS patient_count, " \
                "RANK() OVER (ORDER BY COUNT(t.pik) DESC) AS rank " \
                "FROM healthdb_schema.\"fact_table\" t " \
                "JOIN healthdb_schema.\"time_dim\" s on s.time_key=t.time_key " \
                "GROUP BY s.year, s.quarter " \
               ") subquery " \
               "ORDER BY year, rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Year", "Quarter", "Patient Count"]) 
df = df.set_index(['Year'])

busiest_quarters_yearwise = df 
busiest_quarters_yearwise

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Quarter,Patient Count
Year,,
2018,Q4,6946
2019,Q3,20010
2019,Q1,18371
2019,Q2,15454
2019,Q4,14388
2020,Q1,13194
2020,Q2,3707
2020,Q3,8
2020,Q4,5
